In [ ]:
import pandas as pd

everyday_df = pd.read_csv('/content/Cannabis_Strains_Features.csv')

In [ ]:
everyday_df.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [ ]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def tokenize(doc):
  lemmas = []
    
  doc = nlp(doc)
    
  for token in doc: 
    if ((token.is_stop == False) and (token.is_punct == False) and (token.pos_ != 'PRON')):
      lemmas.append(token.lemma_)
  
  return lemmas

everyday_df['Description_2'] = everyday_df['Description'].apply(tokenize)

In [ ]:
everyday_df['bow_col'] = "string"
for i in range(0, len(everyday_df['Effects'])):
  everyday_df['bow_col'][i] = everyday_df['Effects'][i].split(",") + everyday_df['Flavor'][i].split(",") + everyday_df['Description_2'][i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
everyday_df.head()

,Strain,Type,Rating,Effects,Flavor,Description,Description_2,bow_col
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"[$, 100, OG, 50/50, hybrid, strain, pack, stro...","[Creative, Energetic, Tingly, Euphoric, Relaxe..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"[98, Aloha, White, Widow, especially, potent, ...","[Relaxed, Aroused, Creative, Happy, Energetic,..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"[1024, sativa, dominant, hybrid, breed, Spain,...","[Uplifted, Happy, Relaxed, Energetic, Creative..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"[13, Dawgs, hybrid, G13, Chemdawg, genetic, br...","[Tingly, Creative, Hungry, Relaxed, Uplifted, ..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","[know, Kosher, Tangie, 24k, gold, 60, indica, ...","[Happy, Relaxed, Euphoric, Uplifted, Talkative..."


In [ ]:
everyday_df['bow_col'][0]

['Creative',
 'Energetic',
 'Tingly',
 'Euphoric',
 'Relaxed',
 'Earthy',
 'Sweet',
 'Citrus',
 '$',
 '100',
 'OG',
 '50/50',
 'hybrid',
 'strain',
 'pack',
 'strong',
 'punch',
 'supposedly',
 'refer',
 'strength',
 'high',
 'price',
 'start',
 'show',
 'Hollywood',
 'plant',
 '$',
 '100',
 'OG',
 'tend',
 'produce',
 'large',
 'dark',
 'green',
 'bud',
 'stem',
 'user',
 'report',
 'strong',
 'body',
 'effect',
 'indica',
 'pain',
 'relief',
 'alert',
 'cerebral',
 'feeling',
 'thank',
 'sativa']

In [ ]:
from collections import Counter

word_counts = Counter()
everyday_df['bow_col'].apply(lambda x: word_counts.update(x))
word_counts.most_common(10)

[('strain', 3021),
 ('  ', 2138),
 ('Happy', 1902),
 ('Relaxed', 1758),
 ('Euphoric', 1669),
 ('effect', 1596),
 ('Uplifted', 1535),
 ('indica', 1399),
 ('hybrid', 1312),
 ('Sweet', 1155)]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words=nlp.Defaults.stop_words, max_features=1000)
vect.fit(everyday_df['bow_col'].astype('str'))
dtm = vect.transform(everyday_df['bow_col'].astype('str'))
dtm = pd.DataFrame(dtm.todense(),columns=vect.get_feature_names())
dtm.head()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,10,11,12,13,14,15,18,1st,20,2012,2014,2015,2016,24,25,2nd,30,3rd,40,47,50,55,60,65,70,75,80,90,91,98,99,abate,ability,accent,ace,ache,achieve,act,active,activity,...,useful,user,usher,usually,utilize,valley,vanilla,variation,variety,version,veteran,vibrant,vigorous,violet,wait,want,warm,washington,way,week,weigh,weight,weighted,west,white,wide,widow,win,winner,wonder,woody,work,world,worth,wrap,wreck,xa0,year,yield,zesty
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.347529,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.173267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.122812,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.269103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.353412,0.0,0.450098,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.117527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.620051,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.115618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.154098,0.000000,0.0,0.0,0.0,0.0,0.0,0.151959,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.116128,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.121769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [ ]:
random_pot = ["I want something to take the edge off of my day, really mellow me out and help get me in the zone"]

In [ ]:
new = vect.transform(random_pot)

In [ ]:
nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.        , 1.21833403, 1.22405446,
         1.25974847, 1.27575554, 1.27670641, 1.28326677, 1.29198692]]),
 array([[1652, 1653, 1651, 2318, 1785,  672, 2251,  166, 2068, 1124]]))

In [ ]:
everyday_df['bow_col'][2068]

['Relaxed',
 'Focused',
 'Uplifted',
 'Talkative',
 'Happy',
 'Pine',
 'Earthy',
 'Spicy/Herbal',
 'Suzy',
 'Q',
 'high',
 'CBD',
 'low',
 'thc',
 'strain',
 'piney',
 'taste',
 'help',
 'treat',
 'symptom',
 'little',
 'high',
 'great',
 'strain',
 'add',
 'burgeon',
 'world',
 'CBD',
 'strain',
 'strain',
 'test',
 'upwards',
 '59:1',
 'CBD',
 'THC',
 'hybrid',
 'great',
 'daytime',
 'use',
 'want',
 'relieve',
 'chronic',
 'pain',
 'nausea',
 'arthritis',
 'muscle',
 'spasm',
 'anxiety',
 'psychoactive',
 'effect',
 '\xa0 \xa0\xa0']

In [ ]:
everyday_df['Rating'].value_counts()

4.5    323
4.3    299
4.4    287
4.6    245
4.2    227
5.0    219
4.7    166
4.8    162
4.0    109
4.1    101
4.9     61
0.0     35
3.9     29
3.8     23
3.7     16
3.0     13
3.6     11
3.5     10
3.4      5
3.2      3
3.3      3
3.1      1
2.5      1
1.0      1
2.8      1
Name: Rating, dtype: int64

In [37]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dropout

def create_model(units=32):
  model = Sequential()
  
  model.add(Dense(units, activation="relu", input_dim=1000))
  
  n_nodes_output = 25
  model.add(Dense(n_nodes_output, activation='softmax'))
  
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

#results = model.fit(dtm[:1000], 
#                    everyday_df['Rating'][:1000],
#                    epochs=5,
#                    batch_size=4,
#                    validation_data=(dtm[1000:2000], everyday_df['Rating'][1000:2000]))

model_class = KerasClassifier(build_fn=create_model, verbose=1)

param_grid = {'batch_size': [4, 8],
              'epochs': [5, 6, 9],
              'units':[24, 32]}

grid = GridSearchCV(estimator=model_class, 
                    param_grid=param_grid, 
                    n_jobs=1, 
                    verbose=1)

grid_result = grid.fit(dtm[:1000], everyday_df['Rating'][:1000])

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Epoch 1/5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


200/200 [==============================] - 0s 1ms/step - loss: 3.0141 - accuracy: 0.1100
Epoch 2/5
200/200 [==============================] - 0s 1ms/step - loss: 2.5315 - accuracy: 0.1925
Epoch 3/5
200/200 [==============================] - 0s 1ms/step - loss: 2.3712 - accuracy: 0.3088
Epoch 4/5
200/200 [==============================] - 0s 1ms/step - loss: 2.2438 - accuracy: 0.3288
Epoch 5/5
50/50 [==============================] - 0s 832us/step - loss: 2.5118 - accuracy: 0.2350
Epoch 1/5
200/200 [==============================] - 0s 1ms/step - loss: 2.9559 - accuracy: 0.1125
Epoch 2/5
200/200 [==============================] - 0s 1ms/step - loss: 2.4965 - accuracy: 0.1350
Epoch 3/5
200/200 [==============================] - 0s 1ms/step - loss: 2.3731 - accuracy: 0.3000
Epoch 4/5
200/200 [==============================] - 0s 1ms/step - loss: 2.2633 - accuracy: 0.3938
Epoch 5/5
50/50 [==============================] - 0s 801us/step - loss: 2.5894 - accuracy: 0.1750
Epoch 1/5
200/200 [=

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  2.0min finished


250/250 [==============================] - 0s 1ms/step - loss: 2.9099 - accuracy: 0.1160
Epoch 2/5
250/250 [==============================] - 0s 1ms/step - loss: 2.5002 - accuracy: 0.1790
Epoch 3/5
250/250 [==============================] - 0s 1ms/step - loss: 2.3830 - accuracy: 0.2780
Epoch 4/5
250/250 [==============================] - 0s 1ms/step - loss: 2.2568 - accuracy: 0.3950
Epoch 5/5
250/250 [==============================] - 0s 1ms/step - loss: 2.1127 - accuracy: 0.4400
Best: 0.21299999952316284 using {'batch_size': 4, 'epochs': 5, 'units': 24}
Means: 0.21299999952316284, Stdev: 0.029086077123937806 with: {'batch_size': 4, 'epochs': 5, 'units': 24}
Means: 0.20099999904632568, Stdev: 0.02353720379671924 with: {'batch_size': 4, 'epochs': 5, 'units': 32}
Means: 0.2, Stdev: 0.0380788609117004 with: {'batch_size': 4, 'epochs': 6, 'units': 24}
Means: 0.20199999809265137, Stdev: 0.040693975290792335 with: {'batch_size': 4, 'epochs': 6, 'units': 32}
Means: 0.1929999977350235, Stdev: 

In [ ]:
from keras.models import load_model

model.save('eli_model.tf')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: eli_model.tf/assets
